<a href="https://colab.research.google.com/github/rmcpantoja/My-Colab-Notebooks/blob/main/notebooks/ForwardTacotron_Espa%C3%B1ol_Entrenamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📓 Cuaderno de entrenamiento de `Forward Tacotron` (En español). 📓

*Versión: 1.0.*

---

Este cuaderno ha sido desarrollado por [rmcpantoja](https://github.com/rmcpantoja).

# ✉️ Agradecimientos:

* A [Xx_Nessu_xX](https://fakeyou.com/profile/Xx_Nessu_xX) por el diseño y corrección del cuaderno.
* A [Exink](http://github.com/exink) por la ayuda en el desarrollo de este cuaderno.

## 📝 Créditos:

* Repositorio de [as-ideas/ForwardTacotron](https://github.com/as-ideas/ForwardTacotron).

*Última actualización: 26/03/2023*

In [ ]:
#@markdown ### 👁️Comprobar la GPU asignada.
#@markdown ---
#@markdown Necesitas una `Tesla T4` como mínimo. Si tienes una GPU como `K80`, ve a la barra de menús y selecciona Entorno de ejecución > Desconectarse y eliminar entorno de ejecución.

!nvidia-smi -L

In [ ]:
#@markdown ### 📁Montar Google drive.
#@markdown ---
#@markdown Esto es muy importante para almacenar los puntos de control y los conjuntos de datos procesados con los que Forward Tacotron podrá trabajar. Sin embargo, algunas notas importantes:
#@markdown * Es importante que verifiques tu espacio de almacenamiento en [Drive](http://drive.google.com/). De acuerdo al tamaño del dataset, necesitas calcular una mayor cantidad de espacio disponible.

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@markdown ## 💻Iniciar el proceso de instalación.
#@markdown ---
#@markdown Esto instalará el sintetizador y otras dependencias importantes.

#@markdown * Nota: reinicia el entorno de ejecución si se te solicita y, a continuación, ejecuta esta celda nuevamente y después prosigue más adelante sin problemas.

#@markdown #### <font color=orange>¿Quieres utilizar la versión 3.2 de ForwardTacotron?

#@markdown La versión 3.2 es la última que sólo contiene soporte para un único hablante.
only_singlespeaker_version = True #@param {type:"boolean"}
#@markdown ---
%cd /content
import os
from os.path import exists
if (not os.path.exists("/content/ForwardTacotron")):
  !git clone https://github.com/as-ideas/ForwardTacotron
# pip:
!pip install numba librosa pyworld phonemizer webrtcvad PyYAML dataclasses soundfile scipy tensorboard matplotlib unidecode inflect resemblyzer==0.1.1-dev pandas
!pip install --upgrade gdown
%cd /content/ForwardTacotron
if only_singlespeaker_version:
  !git checkout 632b453c6cb6d15dfe4dd168cd60c60e56731829
!rm -r .git/
#apt:
!apt install espeak-ng
print("Listo")

# 🗂️ Preparación del proyecto.

In [ ]:
%cd /content/ForwardTacotron
#@markdown ### 🔧Asistente de configuración.

#@markdown Estas son algunas configuraciones con las que podremos modificar ajustes relacionados a datos y entrenamiento. Puedes ejecutar esta celda para administrarla.

#@markdown ---

# imports:
import os
import ipywidgets as widgets
from IPython.display import Markdown
from utils.files import read_config, save_config

# interface:
model_type  = widgets.Dropdown(
    options=['Un solo hablante', 'Un solo hablante (versión 3.2)', 'Varios hablantes'],
    value='Un solo hablante (versión 3.2)',
    description='Variante de modelo a usar:',
)

tts_model_id = widgets.Text(
    value='EjemploTTS',
    description='Nombre deseado para el modelo:',
)
tts_model = widgets.ToggleButtons(
    options=['forward_tacotron', 'multi_forward_tacotron', 'fast_pitch'],
    description='Modelo que se va a entrenar:',
)
continue_training = widgets.Checkbox(
    value=False,
    description='¿Continuar un entrenamiento?',
)

preprocess_path = widgets.Text(
    value='/content/drive/MyDrive/ForwardTacotron/EjemploTTS/dataset_preprocessed.zip',
    description='Ubicación del preprocesamiento en Drive (si está activado)',
    disabled=True
)

custom_save_dir = widgets.Checkbox(
    value=False,
    description='¿Guardar puntos de control y preprocesamiento en una ubicación personalizada? (recomendado)',
)
save_dir = widgets.Text(
    value='/content/drive/MyDrive/ForwardTacotron/EjemploTTS',
    description='Si está activado, ¿dónde deseas guardarlo?',
    disabled=True
)
sample_rate = widgets.IntText(
    value=22050,
    min=16000,
    max=48000,
    step=1000,
    description='Frecuencia de muestreo: (Opcional)',
    style={'description_width': 'initial'}
)
metafile_format = widgets.Dropdown(
    options=['ljspeech', 'ljspeech_multi', 'pandas', 'vctk'],
    value='ljspeech',
    description='Formato del archivo de transcripciones:',
    style={'description_width': 'initial'}
)
if only_singlespeaker_version:
    display(Markdown("**Usando la versión 3.2. Por lo tanto, no se podrá seleccionar formato de transcripciones.**"))
    metafile_format.disabled = True
n_val = widgets.IntText(
    value=10,
    min=1,
    max=200,
    step=1,
    description='Número de validaciones (ajústalo según el tamaño del conjunto de datos):',
    style={'description_width': 'initial'}
)

language = widgets.Dropdown(
    options=['es', 'es-419'],
    value='es',
    description='Variación de idioma del conjunto de datos:',
    style={'description_width': 'initial'}
)
# Crea otro campo numérico para escribir cada cuántos pasos se generan avances del entrenamiento
plot_every = widgets.IntText(
    value=1000,
    min=500,
    max=5000,
    step=1000,
    description='Intervalo de pasos para generar señales de entrenamiento del modelo (tensorboard):',
    style={'description_width': 'initial'}
)
applyBTN = widgets.Button(
    description='Aplicar configuración',
    button_style='success'
)

def on_continue_training_change(change):
    if change['new']:
        preprocess_path.disabled = False
    else:
        preprocess_path.disabled = True

def on_custom_save_dir_change(change):
    if change['new']:
        save_dir.disabled = False
    else:
        save_dir.disabled = True

def check_config(model_type):
    if model_type == "Un solo hablante":
        config_path = "configs/singlespeaker.yaml"
    elif model_type == "Varios hablantes":
        config_path = "configs/multispeaker.yaml"
    elif model_type == "Un solo hablante (versión 3.2)" and only_singlespeaker_version:
        config_path = "config.yaml"
    else:
        raise Exception("Tipo de modelo no soportado. Actualmente, puedes elegir entre un solo hablante o varios")
    return config_path

def save_settings(b):
    tts_id = tts_model_id.value
    display(Markdown(f"""
# Resumen de la configuración:

* Tipo de modelo: {model_type.value}.
* Nombre de modelos TTS: {tts_id}.
* Modelo TTS a usar: {tts_model.value}.
* Continuación de un entrenamiento: {continue_training.value}.
* Usar un directorio de guardado personalizado para los modelos: {custom_save_dir.value}.
* Frecuencia de muestreo: {sample_rate.value}.
* Formato de transcripciones: {metafile_format.value}.
* Número de validaciones: {n_val.value}.
* Idioma: {language.value}
* Generación de muestras del entrenamiento cada {plot_every.value} pasos.

Si hay algo en lo que debas arreglar, puedes ajustar y aplicar nuevamente las configuraciones.
    """))
    config_path = check_config(model_type.value)
    config = read_config(config_path)
    config['tts_model_id'] = tts_id
    if tts_model.value == "multi_forward_tacotron" and model_type.value == "Un solo hablante":
        raise Exception("El modelo multi_forward_tacotron solo está soportado en modelos de varios hablantes.")
    elif tts_model.value == "multi_forward_tacotron" and only_singlespeaker_version:
        raise Exception("El modelo multi_forward_tacotron no está soportado en esta versión.")
    config['tts_model'] = tts_model.value
    if continue_training.value:
        !unzip -q "{preprocess_path.value}" -d /content/ForwardTacotron
    if custom_save_dir.value:
        if not os.path.exists(save_dir.value):
            os.makedirs(save_dir.value)
    else:
        print("¡Advertencia! Los avances no se guardarán, solo en la carpeta local del proyecto.")
        save_dir.value = "/content/ForwardTacotron"
    config['dsp']['sample_rate'] = sample_rate.value
    config['dsp']['vad_sample_rate'] = sample_rate.value
    if not only_singlespeaker_version:
        if metafile_format.value == "ljspeech" and model_type.value == "Varios hablantes":
            raise Exception("El formato ljspeech es compatible solamente con los modelos de un solo hablante.")
        elif metafile_format.value == "ljspeech_multi" or metafile_format.value == "pandas" or metafile_format.value == "vctk":
            if model_type.value == "Un solo hablante":
                raise Exception("Ni el modelo ljspeech_multi, pandas ni vctk no son compatibles con modelos para un solo hablante.")
        config['preprocessing']['metafile_format'] = metafile_format.value
    config['preprocessing']['n_val'] = n_val.value
    config['preprocessing']['language'] = language.value
    # set no cleaners:
    cleaner_name = 'no_cleaners'
    config['preprocessing']['cleaner_name'] = cleaner_name
    # reduce workers in dur extraction:
    config['duration_extraction']['num_workers'] = 2
    # Tacotron singlespeaker (80k steps):
    if model_type == "Un solo hablante" or model_type == "Un solo hablante (versión 3.2)":
        config['tacotron']['training']['schedule'] = ['5,  1e-3,  10_000,  32', '3,   1e-4,  20_000,  16', '2,   1e-4,  30_000,  8', '1,   1e-4,  40_000,  8', '5,  1e-3,  50_000,  32', '3,   1e-4,  60_000,  16', '2,   1e-4,  70_000,  8', '1,   1e-4,  80_000,  8']
    # todo: multispeaker pretrained models.
    # plot:
    config['tacotron']['training']['plot_every'] = plot_every.value
    if model_type.value == "Varios hablantes":
        config['multi_forward_tacotron']['training']['plot_every'] = plot_every.value
    else:
        config['forward_tacotron']['training']['plot_every'] = plot_every.value
        config['fast_pitch']['training']['plot_every'] = plot_every.value
    # Manage Spanish pretrained models:
    if model_type.value == "Un solo hablante (versión 3.2)" and only_singlespeaker_version:
        if not continue_training.value:
            if not os.path.exists(save_dir.value+"/checkpoints/"+tts_id+".tacotron"):
                os.makedirs(save_dir.value+"/checkpoints/"+tts_id+".tacotron")
            print(f"Descarga del modelo preentrenado en: {save_dir.value}/checkpoints/{tts_id}.tacotron")
            !gdown -q 1--8jkfZaFk2uDoqX-FE-CI4NUja5NR2V -O "{save_dir.value}/checkpoints/{tts_id}.tacotron/latest_model.pt"
        else:
            print(f"El entrenamiento se retomará en: {save_dir.value}/checkpoints/{tts_id}.tacotron")
    else:
        print(f"¡Advertencia! Actualmente, no existe un modelo preentrenado para la edición {model_type.value}. Es probable que estemos trabajando en ello. Si deseas, puedes enviar una contribución en la sección pull requests en GitHub. Se entrenará un modelo desde cero.")
    # check checkpoints:
    if continue_training.value:
        if custom_save_dir.value:
            if not os.path.exists(save_dir.value+"/checkpoints/"+tts_id+".tacotron/latest_model.pt"):
                raise Exception("Parece que estás intentando continuar un entrenamiento. Sin embargo, no encuentro el modelo en la ruta especificada. Por favor, arréglalo."+save_dir.value+"/checkpoints/"+tts_id+".tacotron/latest_model.pt")
    # phoneme singlespeaker:
    config['preprocessing']['use_phonemes'] = True
    # attention:
    if model_type.value == "Varios hablantes":
        config['multi_forward_tacotron']['training']['filter_attention'] = True
        config['multi_forward_tacotron']['training']['min_attention_sharpness'] = 0.5
        config['multi_forward_tacotron']['training']['min_attention_alignment'] = 0.75
    else:
        config['forward_tacotron']['training']['filter_attention'] = True
        config['forward_tacotron']['training']['min_attention_sharpness'] = 0.5
        config['forward_tacotron']['training']['min_attention_alignment'] = 0.75
    save_config(config, config_path)
    print("¡Configuración guardada con éxito!")
    return config_path

continue_training.observe(on_continue_training_change, names='value')
custom_save_dir.observe(on_custom_save_dir_change, names='value')

display(model_type )
display(tts_model_id)
display(tts_model)
display(Markdown("La opción multi_forward_tacotron está soportada solamente para modelos de varios hablantes."))
display(continue_training)
display(preprocess_path)
display(custom_save_dir)
display(save_dir)
display(sample_rate)
display(metafile_format)
display(Markdown("El formato ljspeech es el único que se usa para modelos de un solo hablante."))
display(n_val)
display(language)
display(Markdown('Tengamos en cuenta que "es" equivale a español de españa (recomendado) y "es-419" equivale al español de latinoamérica.'))
display(plot_every)
display(Markdown("Nota: esta configuración aplicará en todos los modelos: Tacotron, Forward_tacotron, multi_forward_tacotron (Si se entrena con varios hablantes), y FastPitch."))
display(applyBTN)
applyBTN.on_click(save_settings)
config_path = check_config(model_type.value)

In [ ]:
#@markdown ### ⚙️Aplicar parches de acuerdo a la configuración.
#@markdown ---
#@markdown Antes de continuar, es recomendable ejecutar esta celda para parchear las rutas donde se guardan los modelos. Saltando esta celda, estos se guardarán en la carpeta raíz del proyecto en lugar de la carpeta de guardado personalizada (si tienes la casilla marcada correspondiente).

tts_id = tts_model_id.value
voc_id = tts_model_id.value+"_voc"
name = "test"

if only_singlespeaker_version:
  print("Aplicando parche para la versión 3.2...")
  with open('/content/ForwardTacotron/utils/paths.py', 'w') as f:
    f.write('''
import os
from pathlib import Path


class Paths:
    """Manages and configures the paths used by WaveRNN, Tacotron, and the data."""
    def __init__(self, data_path, voc_id, tts_id):
        self.base = Path(__file__).parent.parent.expanduser().resolve()

        # Data Paths
        self.data = Path(data_path).expanduser().resolve()
        self.quant = self.data/'quant'
        self.mel = self.data/'mel'
        self.gta = self.data/'gta'
        self.att_pred = self.data/'att_pred'
        self.alg = self.data/'alg'
        self.raw_pitch = self.data/'raw_pitch'
        self.phon_pitch = self.data/'phon_pitch'
        self.phon_energy = self.data/'phon_energy'
        self.model_output = self.base / 'model_output'
        self.save_dir = Path("'''+save_dir.value+'''").expanduser().resolve()
        self.voc_checkpoints = self.save_dir/'checkpoints/'''+voc_id+'''.wavernn'
        self.voc_top_k = self.voc_checkpoints/'top_k_models'
        self.voc_log = self.voc_checkpoints/'logs'
        self.taco_checkpoints = self.save_dir/'checkpoints/'''+tts_id+'''.tacotron'
        self.taco_log = self.taco_checkpoints / 'logs'
        self.forward_checkpoints = self.save_dir/'checkpoints/'''+tts_id+'''.forward'
        self.forward_log = self.forward_checkpoints/'logs'

        self.create_paths()

    def create_paths(self):
        os.makedirs(self.data, exist_ok=True)
        os.makedirs(self.quant, exist_ok=True)
        os.makedirs(self.mel, exist_ok=True)
        os.makedirs(self.gta, exist_ok=True)
        os.makedirs(self.alg, exist_ok=True)
        os.makedirs(self.att_pred, exist_ok=True)
        os.makedirs(self.raw_pitch, exist_ok=True)
        os.makedirs(self.phon_pitch, exist_ok=True)
        os.makedirs(self.phon_energy, exist_ok=True)
        os.makedirs(self.voc_checkpoints, exist_ok=True)
        os.makedirs(self.voc_top_k, exist_ok=True)
        os.makedirs(self.taco_checkpoints, exist_ok=True)
        os.makedirs(self.forward_checkpoints, exist_ok=True)

    def get_tts_named_weights(self, name):
        """Gets the path for the weights in a named tts checkpoint."""
        return self.taco_checkpoints / f'{name}_weights.pyt'

    def get_tts_named_optim(self, name):
        """Gets the path for the optimizer state in a named tts checkpoint."""
        return self.taco_checkpoints / f'{name}_optim.pyt'

    def get_voc_named_weights(self, name):
        """Gets the path for the weights in a named voc checkpoint."""
        return self.voc_checkpoints/f'{name}_weights.pyt'

    def get_voc_named_optim(self, name):
        """Gets the path for the optimizer state in a named voc checkpoint."""
        return self.voc_checkpoints/f'{name}_optim.pyt'
''')
else:
  print("Aplicando parche para la versión actual...")
  with open('/content/ForwardTacotron/utils/paths.py', 'w') as f:
    f.write('''
import os
from pathlib import Path


class Paths:
    """Manages and configures the paths used by WaveRNN, Tacotron, and the data."""
    def __init__(self, data_path, tts_id):

        # directories
        self.base = Path(__file__).parent.parent.expanduser().resolve()
        self.data = Path(data_path).expanduser().resolve()
        self.quant = self.data/'quant'
        self.mel = self.data/'mel'
        self.gta = self.data/'gta'
        self.att_pred = self.data/'att_pred'
        self.alg = self.data/'alg'
        self.speaker_emb = self.data/'speaker_emb'
        self.mean_speaker_emb = self.data/'mean_speaker_emb'
        self.raw_pitch = self.data/'raw_pitch'
        self.phon_pitch = self.data/'phon_pitch'
        self.phon_energy = self.data/'phon_energy'
        self.model_output = self.base / 'model_output'
        self.save_dir = Path("'''+save_dir.value+'''").expanduser().resolve()
        self.taco_checkpoints = self.save_dir/'checkpoints/'''+tts_id+'''.tacotron'
        self.taco_log = self.taco_checkpoints / 'logs'
        self.forward_checkpoints = self.save_dir/'checkpoints/'''+tts_id+'''.forward'
        self.forward_log = self.forward_checkpoints/'logs'

        # pickle objects
        self.train_dataset = self.data / 'train_dataset.pkl'
        self.val_dataset = self.data / 'val_dataset.pkl'
        self.text_dict = self.data / 'text_dict.pkl'
        self.speaker_dict = self.data / 'speaker_dict.pkl'
        self.att_score_dict = self.data / 'att_score_dict.pkl'
        # future:
        self.duration_stats = self.data / 'duration_stats.pkl'

        self.create_paths()

    def create_paths(self):
        os.makedirs(self.data, exist_ok=True)
        os.makedirs(self.quant, exist_ok=True)
        os.makedirs(self.mel, exist_ok=True)
        os.makedirs(self.gta, exist_ok=True)
        os.makedirs(self.alg, exist_ok=True)
        os.makedirs(self.speaker_emb, exist_ok=True)
        os.makedirs(self.mean_speaker_emb, exist_ok=True)
        os.makedirs(self.att_pred, exist_ok=True)
        os.makedirs(self.raw_pitch, exist_ok=True)
        os.makedirs(self.phon_pitch, exist_ok=True)
        os.makedirs(self.phon_energy, exist_ok=True)
        os.makedirs(self.taco_checkpoints, exist_ok=True)
        os.makedirs(self.forward_checkpoints, exist_ok=True)

    def get_tts_named_weights(self, name):
        """Gets the path for the weights in a named tts checkpoint."""
        return self.taco_checkpoints / f'{name}_weights.pyt'

    def get_tts_named_optim(self, name):
        """Gets the path for the optimizer state in a named tts checkpoint."""
        return self.taco_checkpoints / f'{name}_optim.pyt'

    def get_voc_named_weights(self, name):
        """Gets the path for the weights in a named voc checkpoint."""
        return self.voc_checkpoints/f'{name}_weights.pyt'

    def get_voc_named_optim(self, name):
        """Gets the path for the optimizer state in a named voc checkpoint."""
        return self.voc_checkpoints/f'{name}_optim.pyt'
''')
print("¡Listo!")

## Trabajando con el conjunto de datos.

**Puedes saltarte estas celdas si ya pre-procesaste un dataset por primera vez y quieres entrenarlo en el último punto de control que se haya guardado. De lo contrario, expande esta sección y lee las instrucciones de cada celda.**

In [ ]:
import zipfile
import os
import os.path
#@markdown ### 💾Procesamiento del conjuntos de datos.
#@markdown ---
#@markdown * Nota: si vas a preprocesar conjuntos de datos de mayor tamaño, se recomienda tener más espacio disponible en drive.
#@markdown ---
#@markdown #### 🔊Ruta de los audios. Ellos deberán almacenarse en un archivo .zip:
wavs_path = "/content/drive/MyDrive/wavs.zip" #@param {type:"string"}
#@markdown ---
#@markdown #### ✍️Ruta de transcripción: (Por defecto metadata.csv)
list_path = "/content/drive/MyDrive/list.csv" #@param {type:"string"}
list_filename = os.path.basename(list_path).split('/')[-1]
#@markdown ---
%cd /content
!mkdir dataset
%cd dataset
!mkdir wavs
if zipfile.is_zipfile(wavs_path):
  !unzip -j "$wavs_path" -d /content/dataset/wavs
else:
  print("Aviso: la ruta de audios no es un archivo comprimido.")
if list_path.endswith('.txt'):
  raise Exception("El formato de la transcripción deberá estar en formato csv")

if not os.path.exists(list_path):
  raise Exception("Error: el archivo de transcripción no existe, inténtelo de nuevo por favor.")
else:
  !cp $list_path /content/dataset
%cd /content/ForwardTacotron
print("Ejecutando procesamiento...")
if only_singlespeaker_version:
  !python preprocess.py --path /content/dataset
else:
  !python preprocess.py --path /content/dataset --config "{config_path}" --metafile "{list_filename}"
if custom_save_dir.value:
  print("Respaldando preprocesamiento...")
  if model_type == "Varios hablantes:":
    !zip -r "{save_dir.value}/dataset_preprocessed.zip" data_multisspeaker
  elif model_type == "Un solo hablante" or model_type == "Un solo hablante (versión 3.2)":
    !zip -r "{save_dir.value}/dataset_preprocessed.zip" data
  else:
    raise Exception("No se reconoce el tipo de modelo. Recuerda que solo puedes elegir entre un solo hablante o varios.")
  print("El preprocesamiento se ha comprimido. Esto es útil para reanudar un entrenamiento junto a los puntos de control. Las configuraciones no se guardarán, así que recuerda los ajustes de configuraciones de esta sesión ya que te será útil para retomarlo en cualquier momento.")

### <font color='red'>⚠️ ¡Precaución! Debes ejecutar esta celda si tienes un conjunto de datos en tu Forward Tacotron y quieres entrenar otro. Los contenidos se borrarán. ⚠️ </font>

In [ ]:
#@markdown ### <font color='red'>Borrar el conjunto de datos actual (si existe)
#@markdown ---
#@markdown Debido a que los datasets se encuentran en la carpeta de trabajo, es posible que necesites entrenar otro datasetp. Si es así, ejecuta esta celda para hacerlo.
# conjunto de datos
!rm -rf /content/ForwardTacotron/dataset
# preprocesado:
!rm -rf /content/ForwardTacotron/data/*

# 🏋️¡A entrenar!
Esta serie de pasos requerirán de tiempo para conseguir un entrenamiento estable y tras horas, y a veces algunos días, obtener los resultados finales. Por favor, sugiero leer atentamente las indicaciones de cada una de las celdas.

In [ ]:
#@markdown ### 📈Ejecutar la extensión Tensorboard.
#@markdown ---
#@markdown El tensorboard sirve para visualizar el proceso de entrenamiento del modelo. Ten en cuenta que si quieres visualizar esto, puedes ir a las pestañas **audio**, **image** o **scalars**.
%load_ext tensorboard
print(f"Directorio: {save_dir.value}/checkpoints")
%tensorboard --logdir "{save_dir.value}/checkpoints"
import tensorflow as tf
import datetime

In [ ]:
#@markdown ### 🎤Entrenamiento 1: Tacotron.
#@markdown ---
#@markdown Un punto muy a tomar en cuenta es la división del entrenamiento.
#@markdown * El modelo se entrenará entre un total de 40k pasos. De forma predeterminada, los respaldos se guardan cada 10k pasos, por lo que deberíamos preocuparnos por el almacenamiento. ___(En cambio, puedes borrar los respaldos antiguos. Igualmente, el que realmente importa y se usa es el latest_model que se guarda más a menudo).___
#@markdown * Asimismo, este entrenamiento cumple un cronograma el cual se aplicarán parámetros diferentes.
#@markdown
#@markdown Sin más, ¡a entrenar!.
if only_singlespeaker_version:
  !python train_tacotron.py
else:
  !python train_tacotron.py --config "{config_path}"
# include att score, pitch, att, aligments and more:
if custom_save_dir.value:
  print("Pero antes, respaldando el trabajo que se acaba de hacer...")
  if model_type.value == "Varios hablantes:":
    !zip "{save_dir.value}/dataset_preprocessed.zip" data_multisspeaker
  elif model_type.value == "Un solo hablante" or model_type == "Un solo hablante (versión 3.2)":
    !zip "{save_dir.value}/dataset_preprocessed.zip" data
  else:
    raise Exception("No se reconoce el tipo de modelo. Recuerda que solo puedes elegir entre un solo hablante o varios.")
print("¡Listo!")

In [ ]:
#@markdown ### 🚀Entrenamiento 2: ForwardTacotron.
#@markdown ---
#@markdown Esto entrenará el modelo final para Forward Tacotron, tomando en cuenta el trabajo realizado anteriormente.
#@markdown * Asimismo, cumplirá un cronograma. Por defecto, se entrena hasta 300k pasos, pero puede funcionar con menos.
#@markdown * Recuerda que se tomará en cuenta la atención basándose en el modelo Tacotron. Si se está entrenando con pocos archivos debido a la mala atención (Podemos darnos cuenta de esto durante el entrenamiento), hay un problema en el conjunto de datos. Así que por favor, procura revisarlo, arreglar lo que sea necesario o aumentar más datos.
#@markdown * Puedes ajustar el tamaño del lote (Batch) si no tienes memoria durante la sesión aquí.
batch_size = 24 #@param {type:"integer"}
#@markdown ---
# check TTS_model:
config_path = check_config(model_type.value)
config = read_config(config_path)
if tts_model.value == "forward_tacotron":
    config['forward_tacotron']['training']['schedule'] = ['5e-5,  150_000,  '+str(batch_size), '1e-5,  300_000,  '+str(batch_size)]
elif tts_model.value == "multi_forward_tacotron":
    config['multi_forward_tacotron']['training']['schedule'] = ['5e-5,  500_000,  '+str(batch_size), '1e-5,  600_000,  '+str(batch_size)]
elif tts_model.value == "fast_pitch":
    config['fast_pitch']['training']['schedule'] = ['1e-5,  5_000,  '+str(batch_size), '5e-5,  100_000,  '+str(batch_size), '2e-5,  300_000,  '+str(batch_sice)]
else:
    raise Exception(f"Este modelo TTS no está soportado: {tts_model.value}.")
save_config(config, config_path)
#@markdown ---
if only_singlespeaker_version:
  !python train_forward.py
else:
  !python train_forward.py --config "{config_path}"

# 🧍¿Has terminado de entrenar por hoy?🏠🚶
🔊Prueba el modelo en el cuaderno de síntesis pulsando [aquí.](https://colab.research.google.com/drive/1yHdMGB5H6JG44TAN5BNcv7f95beN3qYZ)